In [839]:
# Importing the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
import sklearn
%matplotlib inline

In [840]:
train=pd.read_csv('train.csv')

In [841]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [842]:
x_train=train[['PassengerId',  'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']]

In [843]:
test=pd.read_csv('test.csv')

In [844]:
print('train dataset shape:{}, \ntest dataset shape: {}'.format(x_train.shape,test.shape))

train dataset shape:(891, 11), 
test dataset shape: (418, 11)


## Combining data

train.set_index('PassengerId',inplace=True)
test.set_index('PassengerId',inplace=True)

In [845]:
data=pd.concat([x_train,test])

In [846]:
data_copy=data.copy()
data_copy.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Missing data handling

In [847]:
data.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [848]:
data.drop(columns='Cabin',inplace=True)

In [849]:
a=(data_copy['Fare']/data_copy['Age']).mean()

In [850]:
b=data[data_copy['Age'].isnull()].index
for i in b:
    data_copy['Age'].loc[i]=round((data_copy['Fare'].loc[i])/a)

C:\Users\Nandhu\miniconda3\envs\study\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [851]:
data_copy['Age'].isnull().sum()

0

In [852]:
data_copy['Embarked'].fillna(value='S',inplace=True)

In [853]:


data_copy[data_copy['Fare'].isna()]['Pclass']

152    3
Name: Pclass, dtype: int64

In [854]:
Fare3class=data_copy[data_copy['Pclass']==3]['Fare'].mean()
data_copy['Fare'].fillna(value=Fare3class,inplace=True)

##  Outlier detection

In [855]:
def Outlier_Detecter(data,feature):
    Q1=data[feature].quantile(0.25)
    Q3=data[feature].quantile(0.75)
    IQR=Q3-Q1
    Lower_lim=Q1-1.5*IQR
    Upper_lim=Q3+1.5*IQR
    return data[(data[feature] < Lower_lim ) & (data[feature] > Upper_lim)]

In [856]:
print(Outlier_Detecter(data_copy,'Age'))

Empty DataFrame
Columns: [PassengerId, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked]
Index: []


In [857]:
print(Outlier_Detecter(data_copy,'Fare'))

Empty DataFrame
Columns: [PassengerId, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked]
Index: []


##  Feature Creation

In [858]:
data_copy['Title']=data_copy['Name'].apply(lambda x:x.split()[1])

In [859]:
data_copy.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs.
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr.


## Categorical to Numerical

In [873]:
#Considering the important features that helps to predict the target variable 
#Pclass
#Sex
#Age
#SibSp
#Parch
#Fare
#Embarked
#Title

In [874]:
data_cn=data_copy[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Title']]

In [875]:
data_cn.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,3,male,22.0,1,0,7.2500,S,Mr.
1,1,female,38.0,1,0,71.2833,C,Mrs.
2,3,female,26.0,0,0,7.9250,S,Miss.
3,1,female,35.0,1,0,53.1000,S,Mrs.
4,3,male,35.0,0,0,8.0500,S,Mr.


### Sex

In [876]:
from sklearn.preprocessing import LabelEncoder,LabelBinarizer

In [877]:
le=LabelEncoder()
data_cn['Sex']=le.fit_transform(data_cn['Sex'])

C:\Users\Nandhu\miniconda3\envs\study\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [878]:
lb=LabelBinarizer()

In [879]:
lbr=lb.fit_transform(data_cn['Embarked'])

In [880]:
lbd=pd.DataFrame(data=lbr,columns=classes)

In [881]:
data_cn.reset_index(inplace=True)

In [882]:
data_cn=pd.concat([data_cn,lbd],axis=1)


In [883]:
lb=LabelBinarizer()

In [884]:
lbr=lb.fit_transform(data_cn['Title'])

In [886]:
lbd=pd.DataFrame(data=lbr,columns=lb.classes_)

In [887]:
data_cn.reset_index(inplace=True)

In [888]:
data_cn=pd.concat([data_cn,lbd],axis=1)


In [890]:
lb=LabelBinarizer()

In [891]:
lbr=lb.fit_transform(data_cn['Title'])

In [893]:
lbd=pd.DataFrame(data=lbr,columns=lb.classes_)

In [895]:
data_cn=pd.concat([data_cn,lbd],axis=1)


In [896]:
lb=LabelBinarizer()

In [897]:
lbr=lb.fit_transform(data_cn['Title'])

In [898]:
lbd=pd.DataFrame(data=lbr,columns=lb.classes_)

In [900]:
data_cn=pd.concat([data_cn,lbd],axis=1)


In [901]:
data_cn.head()

,level_0,index,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,...,"Pelsmaeker,","Planke,",Rev.,"Shawah,","Steen,","Velde,","Walle,",der,the,y
0,0,0,3,1,22.0,1,0,7.2500,S,Mr.,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,38.0,1,0,71.2833,C,Mrs.,...,0,0,0,0,0,0,0,0,0,0
2,2,2,3,0,26.0,0,0,7.9250,S,Miss.,...,0,0,0,0,0,0,0,0,0,0
3,3,3,1,0,35.0,1,0,53.1000,S,Mrs.,...,0,0,0,0,0,0,0,0,0,0
4,4,4,3,1,35.0,0,0,8.0500,S,Mr.,...,0,0,0,0,0,0,0,0,0,0


In [902]:
data_cn.head()

,level_0,index,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,...,"Pelsmaeker,","Planke,",Rev.,"Shawah,","Steen,","Velde,","Walle,",der,the,y
0,0,0,3,1,22.0,1,0,7.2500,S,Mr.,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,38.0,1,0,71.2833,C,Mrs.,...,0,0,0,0,0,0,0,0,0,0
2,2,2,3,0,26.0,0,0,7.9250,S,Miss.,...,0,0,0,0,0,0,0,0,0,0
3,3,3,1,0,35.0,1,0,53.1000,S,Mrs.,...,0,0,0,0,0,0,0,0,0,0
4,4,4,3,1,35.0,0,0,8.0500,S,Mr.,...,0,0,0,0,0,0,0,0,0,0


##  Train Test Split

In [ ]:
# Training Data


train_cn=data_cn.loc[:891]

In [ ]:
# Testing Data


test_cn=data_cn[891:].copy()

In [ ]:
print('given train dataset shape:{}, \ngiven test dataset shape: {}'.format(train.shape,test.shape))

In [ ]:
print('preprocessed train dataset shape:{}, \npreprocessed test dataset shape: {}'.format(train_cn.shape,test_cn.shape))

In [ ]:
# Features

x_train=train_cn[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Title_0', 'Title_1',
       'Title_2', 'Title_3', 'Title_4', 'Title_5', 'Title_6', 'Embarked_C',
       'Embarked_Q', 'Embarked_S']]
x_test=test_cn[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Title_0', 'Title_1',
       'Title_2', 'Title_3', 'Title_4', 'Title_5', 'Title_6', 'Embarked_C',
       'Embarked_Q', 'Embarked_S']]

In [ ]:
# Target Variable

y_train=train['Survived']


In [ ]:

x_train.head()

In [ ]:
x_test.head()

In [ ]:
y_train.head()

## Importing Machine Learning algorithms and Metrics

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

### Logistic Regression Initial

In [ ]:
# Initializing the algorithm
LR=LogisticRegression(max_iter=500)
    
    
# Fitting the data into the algorithm
lr=LR.fit(x_train,y_train)
    
# Predicting the output for test dataset
y_predict=lr.predict(x_test)

  

In [ ]:

data={'PassengerId':range(892,1310),'Survived':y_predict}


pd.DataFrame(data).set_index('PassengerId').to_csv(r'submission files/submission_LG02.csv')

In [ ]:
best_score=pd.read_csv(r'submission files/submission_LG01.csv')

In [ ]:
a=best_score['Survived']-y_predict

In [ ]:
#fig,ax1=plt.subplot(1,1)
sns.distplot(a,bins=30)


In [ ]:
best_score['new']=y_predict

In [ ]:
sns.heatmap(best_score[['Survived','new']].corr(),annot=True)

### Xgboost

In [ ]:
xg=XGBClassifier(verbosity=0)

In [ ]:
xgf=xg.fit(x_train,y_train)

In [ ]:
y_predict=xgf.predict(x_test)

In [ ]:
y_predict

In [ ]:

data={'PassengerId':range(892,1310),'Survived':y_predict}


pd.DataFrame(data).set_index('PassengerId').to_csv(r'submission files/submission_XGB01.csv')

In [ ]:
best_score=pd.read_csv(r'submission files/submission_LG01.csv')

In [ ]:
a=best_score['Survived']-y_predict

In [ ]:
#fig,ax1=plt.subplot(1,1)
sns.distplot(a,bins=30)


In [ ]:
best_score['new']=y_predict

In [ ]:
sns.heatmap(best_score[['Survived','new']].corr(),annot=True)

### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfs=RandomForestRegressor()

In [ ]:
svr=rfs.fit(x_train,y_train)

In [ ]:
y_predict=svr.predict(x_test)

In [ ]:
y_predict

In [ ]:

data={'PassengerId':range(892,1310),'Survived':y_predict}


pd.DataFrame(data).set_index('PassengerId').to_csv(r'submission files/submission_RF01.csv')

In [ ]:
best_score=pd.read_csv(r'submission files/submission_LG01.csv')

In [ ]:
a=best_score['Survived']-y_predict

In [ ]:
#fig,ax1=plt.subplot(1,1)
sns.distplot(a,bins=30)


In [ ]:
best_score['new']=y_predict

In [ ]:
sns.heatmap(best_score[['Survived','new']].corr(),annot=True)

### Support Vector Regressor

In [ ]:
from sklearn.svm import SVR

In [ ]:
svr=SVR()

In [ ]:
svr=svr.fit(x_train,y_train)

In [ ]:
y_predict=svr.predict(x_test)

In [ ]:
y_predict

In [ ]:

data={'PassengerId':range(892,1310),'Survived':y_predict}


pd.DataFrame(data).set_index('PassengerId').to_csv(r'submission files/submission_SVM01.csv')

In [ ]:
best_score=pd.read_csv(r'submission files/submission_LG01.csv')

In [ ]:
a=best_score['Survived']-y_predict

In [ ]:
#fig,ax1=plt.subplot(1,1)
sns.distplot(a,bins=30)


In [ ]:
best_score['new']=y_predict

In [ ]:
sns.heatmap(best_score[['Survived','new']].corr(),annot=True)